In [ ]:
from matplotlib import pyplot as plt

# Physical dimensions
WHEEL_RADIUS = 0.056 # meters
WHEEL_SEP = 0.222 # meters

class RobotController:
    
    def __init__(self,Kp=0.2,Ki=0.0,wheel_radius=WHEEL_RADIUS, wheel_sep=WHEEL_SEP):
        
        self.Kp = Kp
        self.Ki = Ki
        self.r = wheel_radius
        self.l = wheel_sep
        self.e_sum_l = 0
        self.e_sum_r = 0
        
    def p_control(self, w_desired, w_measured, e_sum, dt):
        error = w_desired - w_measured
        p_term = self.Kp * error
        i_term = self.Ki * e_sum * dt  # Note: Integrate error over time
        duty_cycle = min(max(-1, p_term + i_term), 1)
        e_sum = e_sum + error * dt  # Accumulate error over actual time
        
        return duty_cycle, e_sum
        
        
    def drive(self, v_desired, w_desired, wl, wr, dt):
        
        wl_desired = (v_desired + self.l*w_desired/2)/self.r
        wr_desired = (v_desired - self.l*w_desired/2)/self.r
        
        duty_cycle_l,self.e_sum_l = self.p_control(wl_desired, wl, self.e_sum_l, dt)
        duty_cycle_r,self.e_sum_r = self.p_control(wr_desired, wr, self.e_sum_r, dt)
        
        return duty_cycle_l, duty_cycle_r


class TestData:
    def __init__(self, v_desired, w_desired, v_actual, w_actual, dt):
        self.v_desired = v_desired
        self.w_desired = w_desired
        self.v_actual = v_actual
        self.w_actual = w_actual
        self.dt = dt
        
# Parameters
Kp = 0.2
Ki = 0.0

# Initialising
controller = RobotController(Kp, Ki)

poses = []
velocities = []
duty_cycle_commands = []

data = [
    TestData(0.5, 0.0, 1.24, 1.75, 0.1),
    TestData( 0.50, 0.0, 1.27, 1.66, 0.12),
    TestData( 0.50,  0.0, 1.38,  1.76, 0.1),
    TestData( 0.50, 0.0, 1.38, 1.76, 0.12),
]

for i in data:
    duty_cycle_l, duty_cycle_r = controller.drive(i.v_desired, i.w_desired, i.v_actual, i.w_actual, i.dt)
    print(f"duty cycle l: {duty_cycle_l}, r: {duty_cycle_r}")
    duty_cycle_commands.append([duty_cycle_l,duty_cycle_r])
    
plt.figure()
plt.plot(duty_cycle_commands)
plt.title("Duty cycle commands")
plt.show()


In [ ]:






# Example: Move forward at 0.5 m/s with no rotation
v_desired = 0.5
w_desired = 0.1